# Amazon SageMaker Clarify Model Explainability Monitor - JSON Format

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

---

## Runtime

This notebook takes approximately 60 minutes to run.

## Contents

* [Introduction](#Introduction)
* [General Setup](#General-Setup)
    * [Imports](#Imports)
    * [Handful of configuration](#Handful-of-configuration)
    * [Model file and data files](#Model-file-and-data-files)
* [Real-time Inference Endpoint](#Real-time-Inference-Endpoint)
    * [Deploy the model to an endpoint](#Deploy-the-model-to-an-endpoint)
    * [Invoke the endpoint](#Invoke-the-endpoint)
        * [Example: Single record](#Example:-Single-record)
        * [Example: Two records](#Example:-Two-records)
    * [View captured data](#View-captured-data)
    * [Start generating some artificial traffic](#Start-generating-some-artificial-traffic)
* [Model Explainability Monitor](#Model-Explainability-Monitor)
    * [Baselining job](#Baselining-job)
        * [Configurations](#Configurations)
        * [Kick off baselining job](#Kick-off-baselining-job)
    * [Monitoring Schedule](#Monitoring-Schedule)
        * [Wait for the first execution](#Wait-for-the-first-execution)
        * [Wait for the execution to finish](#Wait-for-the-execution-to-finish)
        * [Inspect execution results](#Inspect-execution-results)
* [Cleanup](#Cleanup)

## Introduction

[Amazon SageMaker Model Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html) continuously monitors the quality of Amazon SageMaker machine learning models in production. It enables developers to set alerts for when there are deviations in the model quality. Early and pro-active detection of these deviations enables corrective actions, such as retraining models, auditing upstream systems, or fixing data quality issues without having to monitor models manually or build additional tooling. 

[Amazon SageMaker Clarify Model Explainability Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-feature-attribution-drift.html) is a model monitor that helps data scientists and ML engineers monitor predictions for feature attribution drift on a regular basis. A drift in the distribution of live data for models in production can result in a corresponding drift in the feature attribution values. As the model is monitored, customers can view exportable reports and graphs detailing feature attributions in SageMaker Studio and configure alerts in Amazon CloudWatch to receive notifications if it is detected that the attribution values drift beyond a certain threshold. 

This notebook demonstrates the process for setting up a model monitor for continuous monitoring of feature attribution drift of a [SageMaker real-time inference endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html). The model input and output are in [SageMaker JSON Lines dense format](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html#common-in-formats). SageMaker Clarify model monitor also supports analyzing CSV data, which is illustrated in [another notebook](https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker_model_monitor/fairness_and_explainability/SageMaker-Model-Monitor-Fairness-and-Explainability.ipynb).

In general, you can use the model explainability monitor for real-time inference endpoint in this way,

1. Enable the endpoint for data capture. Then, when the customer invokes the endpoint, the endpoint saves the invocations to a data capture S3 location. 
1. Schedule a model explainability monitor to monitor the endpoint (to be more specific, the data capture S3 location) and a ground truth S3 location.

The monitor executes processing jobs regularly to do feature attribution analysis, and then generate analysis reports and publish metrics to CloudWatch.

## General Setup

The notebook uses the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk). The following cell upgrades the SDK and its dependencies. Then you may need to restart the kernel and rerun the notebook to pick up the up-to-date APIs, if the notebook is executed in the SageMaker Studio.

In [1]:
!pip install -U sagemaker
!pip install -U boto3
!pip install -U botocore


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


### Imports

The following cell imports the APIs to be used by the notebook.

In [2]:
import sagemaker
import pandas as pd
import copy
import datetime
import json
import random
import threading
import time
import pprint

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/zicanl/.config/sagemaker/config.yaml


### Handful of configuration

To begin, ensure that these prerequisites have been completed.

* Specify an AWS Region to host the model.
* Specify an IAM role to execute jobs.
* Define the S3 URIs that stores the model file, input data and output data. For demonstration purposes, this notebook uses the same bucket for them. In reality, they could be separated with different security policies.

In [3]:
sagemaker_session = sagemaker.Session()

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

role = sagemaker.get_execution_role()
print(f"RoleArn: {role}")

# A different bucket can be used, but make sure the role for this notebook has
# the s3:PutObject permissions. This is the bucket into which the data is captured
bucket = sagemaker_session.default_bucket()
print(f"Demo Bucket: {bucket}")
prefix = sagemaker.utils.unique_name_from_base("sagemaker/DEMO-ClarifyModelMonitor")
print(f"Demo Prefix: {prefix}")
s3_key = f"s3://{bucket}/{prefix}"
print(f"Demo S3 key: {s3_key}")

data_capture_s3_uri = f"{s3_key}/data-capture"
baselining_output_s3_uri = f"{s3_key}/baselining-output"
monitor_output_s3_uri = f"{s3_key}/monitor-output"

print(f"The endpoint will save the captured data to: {data_capture_s3_uri}")
print(f"The baselining job will save the analysis results to: {baselining_output_s3_uri}")
print(f"The monitor will save the analysis results to: {monitor_output_s3_uri}")

AWS region: us-west-2
RoleArn: arn:aws:iam::678264136642:role/Admin
Demo Bucket: sagemaker-us-west-2-678264136642
Demo Prefix: sagemaker/DEMO-ClarifyModelMonitor-1705692269-8d04
Demo S3 key: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692269-8d04
The endpoint will save the captured data to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692269-8d04/data-capture
The baselining job will save the analysis results to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692269-8d04/baselining-output
The monitor will save the analysis results to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692269-8d04/monitor-output


### Model file and data files

This example includes a prebuilt [SageMaker Linear Learner](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html) model trained by [a SageMaker Clarify offline processing example notebook](https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-clarify/fairness_and_explainability/fairness_and_explainability_jsonlines_format.ipynb). The model supports [SageMaker JSON Lines dense format](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html#common-in-formats) (MIME type `"application/jsonlines"`).

* The model input can one or more lines, each line is a JSON object that has a "features" key pointing to a list of feature values concerning demographic characteristics of individuals. For example,

```
{"features":[28,2,133937,9,13,2,0,0,4,1,15024,0,55,37]}
{"features":[43,2,72338,12,14,2,12,0,1,1,0,0,40,37]}
```

* The model output has the predictions of whether a person has a yearly income that is more than $50,000. Each prediction is a JSON object that has a "predicted_label" key pointing to the predicted label, and the "score" key pointing to the confidence score. For example,

```
{"predicted_label":1,"score":0.989977359771728}
{"predicted_label":1,"score":0.504138827323913}
```

In [4]:
model_file = "model/ll-adult-prediction-model.tar.gz"

This example includes two dataset files, both in the JSON format. The data also originates from [the SageMaker Clarify offline processing example notebook](https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-clarify/fairness_and_explainability/fairness_and_explainability_jsonlines_format.ipynb).

In [5]:
train_dataset_path = "test_data/validation-dataset.json"
test_dataset_path = "test_data/test-dataset.json"
dataset_type = "application/json"

The train dataset has the features and the ground truth label (pointed to by the key "label"),

In [6]:
!head -n 5 $train_dataset_path

{"instances":[{"features":[41,2,220531,14,15,2,9,0,4,1,0,0,60,38],"label":1},{"features":[33,2,35378,9,13,2,11,5,4,0,0,0,45,38],"label":1},{"features":[36,2,223433,12,14,2,11,0,4,1,7688,0,50,38],"label":1},{"features":[40,2,220589,7,12,4,0,1,4,0,0,0,40,38],"label":0},{"features":[30,2,231413,15,10,2,2,0,4,1,0,0,40,38],"label":1},{"features":[33,4,218164,11,9,2,2,0,4,1,0,0,40,38],"label":0},{"features":[42,2,213464,15,10,2,2,0,4,1,0,0,40,38],"label":0},{"features":[20,2,247794,11,9,4,11,1,4,0,0,0,84,38],"label":0},{"features":[43,2,174575,15,10,0,0,1,4,1,0,0,45,38],"label":0},{"features":[42,4,54202,14,15,2,9,0,4,1,0,0,50,38],"label":1},{"features":[27,2,126060,11,9,4,3,1,4,0,0,0,40,38],"label":0},{"features":[25,2,182866,11,9,4,5,3,4,1,0,0,40,38],"label":0},{"features":[43,2,302041,11,9,4,0,1,2,0,0,0,40,38],"label":0},{"features":[30,2,91145,11,9,4,5,4,4,1,0,0,55,38],"label":0},{"features":[41,2,648223,3,2,3,4,4,4,1,0,0,40,25],"label":0},{"features":[60,2,101096,10,16,4,9,1,4,0,0,0,65,

The test dataset only has features.

In [ ]:
!head -n 5 $test_dataset_path

Here are the headers of the train dataset. "Target" is the header of the ground truth label, and the others are the feature headers. They will be used to beautify the analysis report.

In [ ]:
all_headers = [
    "Age",
    "Workclass",
    "fnlwgt",
    "Education",
    "Education-Num",
    "Marital Status",
    "Occupation",
    "Relationship",
    "Ethnic group",
    "Sex",
    "Capital Gain",
    "Capital Loss",
    "Hours per week",
    "Country",
    "Target",
]
label_header = all_headers[-1]

To verify that the execution role for this notebook has the necessary permissions to proceed, put a simple test object into the S3 bucket specified above. If this command fails, update the role to have `s3:PutObject` permission on the bucket and try again.

In [ ]:
sagemaker.s3.S3Uploader.upload_string_as_file_body(
    body="hello",
    desired_s3_uri=f"{s3_key}/upload-test-file.txt",
    sagemaker_session=sagemaker_session,
)
print("Success! We are all set to proceed with uploading to S3.")

Then upload the files to S3 so that they can be used by SageMaker.

In [ ]:
model_url = sagemaker.s3.S3Uploader.upload(
    local_path=model_file,
    desired_s3_uri=s3_key,
    sagemaker_session=sagemaker_session,
)
print(f"Model file has been uploaded to {model_url}")

train_data_s3_uri = sagemaker.s3.S3Uploader.upload(
    local_path=train_dataset_path,
    desired_s3_uri=s3_key,
    sagemaker_session=sagemaker_session,
)
print(f"Train data is uploaded to: {train_data_s3_uri}")
test_data_s3_uri = sagemaker.s3.S3Uploader.upload(
    local_path=test_dataset_path,
    desired_s3_uri=s3_key,
    sagemaker_session=sagemaker_session,
)
print(f"Test data is uploaded to: {test_data_s3_uri}")

## Real-time Inference Endpoint

This section creates a SageMaker real-time inference endpoint to showcase the data capture capability in action. The model monitor will be scheduled for the endpoint and process the captured data.


### Deploy the model to an endpoint

Start with deploying the pre-trained model. Here, create a SageMaker `Model` object with the inference image and model file. Then deploy the model with the data capture configuration and wait until the endpoint is ready to serve traffic.

[DataCaptureConfig](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_monitor.data_capture_config.DataCaptureConfig) enables capturing the request payload and the response payload of the endpoint. Payloads are typically treated as binary data and encoded in BASE64 by default, allowing them to be stored in capture data files. However, by specifying the data format in the `json_content_types` parameter as shown below, the payloads can be captured as plain text instead.

In [ ]:
model_name = sagemaker.utils.unique_name_from_base("DEMO-ll-adult-pred-model-monitor")
endpoint_name = model_name
print(f"SageMaker model name: {model_name}")
print(f"SageMaker endpoint name: {endpoint_name}")

image_uri = sagemaker.image_uris.retrieve("linear-learner", region, "1")
print(f"SageMaker Linear Learner image: {image_uri}")

model = sagemaker.model.Model(
    role=role,
    name=model_name,
    image_uri=image_uri,
    model_data=model_url,
    sagemaker_session=sagemaker_session,
)

data_capture_config = sagemaker.model_monitor.DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,  # Capture 100% of the traffic
    destination_s3_uri=data_capture_s3_uri,
    json_content_types=[dataset_type],
)

**NOTE**: The following cell takes about 10 minutes to deploy the model.

In [ ]:
print(f"Deploying model {model_name} to endpoint {endpoint_name}")
model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

### Invoke the endpoint

Now send data to this endpoint to get inferences in real time. The model supports mini-batch predictions, so you can put one or more records to a single request.

In [ ]:
with open(test_dataset_path, "r") as f:
    test_data = json.load(f)

#### Example: Single record

Request payload:

In [ ]:
request_payload = {"instances": [test_data["instances"][0]]}
print(json.dumps(request_payload))

Response payload:

In [ ]:
response = sagemaker_session.sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=dataset_type,
    Accept=dataset_type,
    Body=json.dumps(request_payload),
)
response_payload = response["Body"].read().decode("utf-8")
response_payload

#### Example: Two records

Request payload:

In [ ]:
request_payload["instances"] = test_data["instances"][:2]
request_payload

Response payload:

In [ ]:
response = sagemaker_session.sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=dataset_type,
    Accept=dataset_type,
    Body=json.dumps(request_payload),
)
response_payload = response["Body"].read().decode("utf-8")
response_payload

### View captured data

Because data capture is enabled in the previous steps, the request and response payload, along with some additional metadata, are saved in the Amazon S3 location specified in the [DataCaptureConfig](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_monitor.data_capture_config.DataCaptureConfig).

Now list the captured data files stored in Amazon S3. There should be different files from different time periods organized based on the hour in which the invocation occurred. The format of the Amazon S3 path is:

`s3://{data_capture_s3_uri}/{endpoint_name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [ ]:
print("Waiting for captured data to show up", end="")
for _ in range(120):
    captured_data_files = sorted(
        sagemaker.s3.S3Downloader.list(
            s3_uri=f"{data_capture_s3_uri}/{endpoint_name}",
            sagemaker_session=sagemaker_session,
        )
    )
    if captured_data_files:
        break
    print(".", end="", flush=True)
    time.sleep(1)
print()
print("Found capture data files:")
print("\n ".join(captured_data_files[-5:]))

Next, view the content of a single capture file.

In [ ]:
captured_data = sagemaker.s3.S3Downloader.read_file(
    s3_uri=captured_data_files[-1],
    sagemaker_session=sagemaker_session,
)
print(captured_data)

Finally, the contents of a single line is present below in formatted JSON to observe a little better.

* `captureData` has two fields, `endpointInput` has the captured invocation request, and `endpointOutput` has the response.
* `eventMetadata` has the inference ID and event ID.

In [ ]:
print(json.dumps(json.loads(captured_data.splitlines()[-1]), indent=4))

### Start generating some artificial traffic
The cell below starts a thread to send some traffic to the endpoint. If there is no traffic, the monitoring jobs are marked as `Failed` since there is no data to process.

Notice the `InferenceId` attribute used to invoke, in this example, it will be used to join the captured data with the ground truth data. If it is not available, then the `eventId` will be used for the join operation.

In [ ]:
class WorkerThread(threading.Thread):
    def __init__(self, do_run, *args, **kwargs):
        super(WorkerThread, self).__init__(*args, **kwargs)
        self.__do_run = do_run
        self.__terminate_event = threading.Event()

    def terminate(self):
        self.__terminate_event.set()

    def run(self):
        while not self.__terminate_event.is_set():
            self.__do_run(self.__terminate_event)

In [ ]:
def invoke_endpoint(terminate_event):
    for index, record in enumerate(test_data["instances"]):
        response = sagemaker_session.sagemaker_runtime_client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType=dataset_type,
            Accept=dataset_type,
            Body=json.dumps({"instances": [record]}),
            InferenceId=str(index),  # unique ID per row
        )
        response["Body"].read()
        time.sleep(1)
        if terminate_event.is_set():
            break


# Keep invoking the endpoint with test data
invoke_endpoint_thread = WorkerThread(do_run=invoke_endpoint)
invoke_endpoint_thread.start()

## Model Explainability Monitor

Similar to the other monitoring types, the standard procedure of creating a [feature attribution drift monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-feature-attribution-drift.html) is first run a baselining job, and then schedule the monitor.

In [ ]:
model_explainability_monitor = sagemaker.model_monitor.ModelExplainabilityMonitor(
    role=role,
    sagemaker_session=sagemaker_session,
    max_runtime_in_seconds=3600,
)

### Baselining job

A baselining job runs predictions on training dataset and suggests constraints. The `suggest_baseline()` method of `ModelExplainabilityMonitor` starts a SageMaker Clarify processing job to generate the constraints.

The step is not mandatory, but providing constraints file to the monitor can enable violations file generation.

#### Configurations

Information about the input data need to be provided to the processor.

`DataConfig` stores information about the dataset to be analyzed. For example, the dataset file and its format (like JSON Lines), where to store the analysis results. Some special things to note about this configuration for the JSON Lines dataset,

* The parameter value `"features"` or `"label"` is **NOT** a header string. Instead, it is a `JMESPath` expression ([refer to its specification](https://jmespath.org/specification.html)) that is used to locate the features list or the ground truth label in the dataset (the ground truth label is not needed for the explainability analysis, the parameter is specified so that the job knows it should be excluded from the dataset). In this example notebook they happen to be the same as the keys in the dataset. But for example, if the dataset has records like below, then the `features` parameter should use value `"data.features.values"`, and the `label` parameter should use value `"data.label"`.

  ```
  {"data": {"features": {"values": [25, 2, 226802, 1, 7, 4, 6, 3, 2, 1, 0, 0, 40, 37]}, "label": 0}}
  ```

* SageMaker Clarify processing job will load the JSON Lines dataset into tabular representation for further analysis, and the parameter `headers` is the list of column names. **The label header shall be the last one in the headers list**, and the order of feature headers shall be the same as the order of features in a record.

In [ ]:
features_jmespath = "instances[*].features"
ground_truth_label_jmespath = "instances[*].label"
data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=train_data_s3_uri,
    s3_output_path=baselining_output_s3_uri,
    features=features_jmespath,
    label=ground_truth_label_jmespath,
    headers=all_headers,
    dataset_type=dataset_type,
)

`ModelConfig` is configuration related to model to be used for inferencing. In order to compute SHAP values, the SageMaker Clarify explainer generates synthetic dataset and then get its predictions for the SageMaker model. To accomplish this, the processing job will use the model to create an ephemeral endpoint (also known as "shadow endpoint"). The processing job will delete the shadow endpoint after the computations are completed. One special thing to note about this configuration for the JSON Lines model input and output,

* `content_template` is used by SageMaker Clarify processing job to convert the tabular data to the request payload acceptable to the shadow endpoint. To be more specific, the placeholder `$features` will be replaced by **the features list** from records. The request payload of a record from the testing dataset happens to be similar to the record itself, like `{"features":[28,2,133937,9,13,2,0,0,4,1,15024,0,55,37]}`, because both the dataset and the model input conform to the same format.

In [ ]:
model_config = sagemaker.clarify.ModelConfig(
    model_name=model_name,  # The name of the SageMaker model
    instance_type="ml.m5.xlarge",  # The instance type of the shadow endpoint
    instance_count=1,  # The instance count of the shadow endpoint
    content_type=dataset_type,  # The data format of the model input
    accept_type=dataset_type,  # The data format of the model output
    content_template='{"instances":$records}',
    record_template='{"features":$features}',
)

Currently, the SageMaker Clarify explainer offers a scalable and efficient implementation of SHAP, so the explainability config is `SHAPConfig`, including

* `baseline`: A list of records (at least one) to be used as the baseline dataset in the Kernel SHAP algorithm, each record is JSON object that includes a list of features. It can also be a S3 object URI, the S3 file should be in the same format as dataset.
* `num_samples`: Number of samples to be used in the Kernel SHAP algorithm. This number determines the size of the generated synthetic dataset to compute the SHAP values.
* `agg_method`: Aggregation method for global SHAP values. Valid values are
  * "mean_abs" (mean of absolute SHAP values for all instances),
  * "median" (median of SHAP values for all instances) and
  * "mean_sq" (mean of squared SHAP values for all instances).
* `use_logit`: Indicator of whether the logit function is to be applied to the model predictions. Default is False. If "use_logit" is true then the SHAP values will have log-odds units.
* `save_local_shap_values`: Indicator of whether to save the local SHAP values in the output location. Default is True.

In [ ]:
# Here use the mean value of train dataset as SHAP baseline
dataset = []
with open(train_dataset_path) as f:
    instances = json.load(f)["instances"]
    for instance in instances:
        dataset.append(instance["features"])
mean_values = pd.DataFrame(dataset).mean().round().astype(int).to_list()
mean_record = {"features": mean_values}
shap_baseline = {"instances": [mean_record]}
print(f"SHAP baseline: {shap_baseline}")

shap_config = sagemaker.clarify.SHAPConfig(
    baseline=shap_baseline,
    num_samples=100,
    agg_method="mean_abs",
    save_local_shap_values=False,
)

#### Kick off baselining job

Call the `suggest_baseline()` method to start the baselining job. The model output has a key "score" pointing to a confidence score value between `0` and `1`. So, the `model_scores` parameter is set to the `JMESPath` expression "score" which can locate the score in the model output.

In [ ]:
confidence_score_jmespath = "predictions[*].score"
model_explainability_monitor.suggest_baseline(
    explainability_config=shap_config,
    data_config=data_config,
    model_config=model_config,
    model_scores=confidence_score_jmespath,  # The JMESPath to locate the confidence score in model output
)

**NOTE**: The following cell waits until the baselining job is completed (in about 10 minutes). It then inspects the suggested constraints. This step can be skipped, because the monitor to be scheduled will automatically pick up baselining job name and wait for it before monitoring execution.

In [ ]:
model_explainability_monitor.latest_baselining_job.wait(logs=False)
print()
model_explainability_constraints = model_explainability_monitor.suggested_constraints()
print(f"Suggested constraints: {model_explainability_constraints.file_s3_uri}")
print(
    sagemaker.s3.S3Downloader.read_file(
        s3_uri=model_explainability_constraints.file_s3_uri,
        sagemaker_session=sagemaker_session,
    )
)

### Monitoring Schedule

With above constraints collected, now call `create_monitoring_schedule()` method to schedule an hourly model explainability monitor.

If a baselining job has been submitted, then the monitor object will automatically pick up the analysis configuration from the baselining job. But if the baselining step is skipped, or if the capture dataset has different nature than the training dataset, then analysis configuration has to be provided.

`ModelConfig` is required by `ExplainabilityAnalysisConfig` for the same reason as it is required by the baselining job. Note that only features are required for computing feature attribution, so ground truth label should be excluded.

Highlights,

* From `endpoint_name` the monitor can figure out the location of data captured by the endpoint.
* `features_attribute` is the `JMESPath` expression to locate the features in model input, similar to the `features` parameter of `DataConfig`.
* `inference_attribute` stores the `JMESPath` expression to locate the confidence score in model output, similar to the `model_scores` parameter of the `suggest_baseline()` method.

In [ ]:
schedule_expression = sagemaker.model_monitor.CronExpressionGenerator.hourly()

In [ ]:
# Remove label because only features are required for the analysis
headers_without_label_header = copy.deepcopy(all_headers)
headers_without_label_header.remove(label_header)
model_explainability_analysis_config = sagemaker.model_monitor.ExplainabilityAnalysisConfig(
    explainability_config=shap_config,
    model_config=model_config,
    headers=headers_without_label_header,
)
model_explainability_monitor.create_monitoring_schedule(
    analysis_config=model_explainability_analysis_config,
    endpoint_input=sagemaker.model_monitor.EndpointInput(
        endpoint_name=endpoint_name,
        destination="/opt/ml/processing/input/endpoint",
        features_attribute=features_jmespath,
        inference_attribute=confidence_score_jmespath,
    ),
    output_s3_uri=monitor_output_s3_uri,
    schedule_cron_expression=schedule_expression,
)
print(
    f"Model explainability monitoring schedule: {model_explainability_monitor.monitoring_schedule_name}"
)

#### Wait for the first execution

The schedule starts jobs at the previously specified intervals. Code below waits until time crosses the hour boundary (in UTC) to see executions kick off.

Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule executions. The execution might start in anywhere from zero to ~20 minutes from the hour boundary. This is expected and done for load balancing in the backend.

In [ ]:
def wait_for_execution_to_start(model_monitor):
    print(
        "An hourly schedule was created above and it will kick off executions ON the hour (plus 0 - 20 min buffer)."
    )

    print("Waiting for the first execution to happen", end="")
    schedule_desc = model_monitor.describe_schedule()
    while "LastMonitoringExecutionSummary" not in schedule_desc:
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        time.sleep(60)
    print()
    print("Done! Execution has been created")

    print("Now waiting for execution to start", end="")
    while schedule_desc["LastMonitoringExecutionSummary"]["MonitoringExecutionStatus"] in "Pending":
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        time.sleep(10)

    print()
    print("Done! Execution has started")

**NOTE**: The following cell waits until the first monitoring execution is started. As explained above, the wait could take more than 60 minutes.

In [ ]:
wait_for_execution_to_start(model_explainability_monitor)

In real world, a monitoring schedule is supposed to be active all the time. But in this example, it can be stopped to avoid incurring extra charges. A stopped schedule will not trigger further executions, but the ongoing execution will continue. And if needed, the schedule can be restarted by `start_monitoring_schedule()`.

In [ ]:
model_explainability_monitor.stop_monitoring_schedule()

#### Wait for the execution to finish

In the previous cell, the first execution has started. This section waits for the execution to finish so that its analysis results are available. Here are the possible terminal states and what each of them mean:

* `Completed` - This means the monitoring execution completed, and no issues were found in the violations report.
* `CompletedWithViolations` - This means the execution completed, but constraint violations were detected.
* `Failed` - The monitoring execution failed, maybe due to client error (perhaps incorrect role permissions) or infrastructure issues. Further examination of `FailureReason` and `ExitMessage` is necessary to identify what exactly happened.
* `Stopped` - job exceeded max runtime or was manually stopped.

In [ ]:
# Waits for the schedule to have last execution in a terminal status.
def wait_for_execution_to_finish(model_monitor):
    schedule_desc = model_monitor.describe_schedule()
    execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
    if execution_summary is not None:
        print("Waiting for execution to finish", end="")
        while execution_summary["MonitoringExecutionStatus"] not in [
            "Completed",
            "CompletedWithViolations",
            "Failed",
            "Stopped",
        ]:
            print(".", end="", flush=True)
            time.sleep(60)
            schedule_desc = model_monitor.describe_schedule()
            execution_summary = schedule_desc["LastMonitoringExecutionSummary"]
        print()
        print(f"Done! Execution Status: {execution_summary['MonitoringExecutionStatus']}")
    else:
        print("Last execution not found")

**NOTE**: The following cell takes about 10 minutes.

In [ ]:
wait_for_execution_to_finish(model_explainability_monitor)

#### Inspect execution results

List the generated reports,

* analysis.json includes the global SHAP values.
* report.* files are static report files to visualize the SHAP values.

In [ ]:
schedule_desc = model_explainability_monitor.describe_schedule()
execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
if execution_summary and execution_summary["MonitoringExecutionStatus"] in [
    "Completed",
    "CompletedWithViolations",
]:
    last_model_explainability_monitor_execution = model_explainability_monitor.list_executions()[-1]
    last_model_explainability_monitor_execution_report_uri = (
        last_model_explainability_monitor_execution.output.destination
    )
    print(f"Report URI: {last_model_explainability_monitor_execution_report_uri}")
    last_model_explainability_monitor_execution_report_files = sorted(
        sagemaker.s3.S3Downloader.list(
            s3_uri=last_model_explainability_monitor_execution_report_uri,
            sagemaker_session=sagemaker_session,
        )
    )
    print("Found Report Files:")
    print("\n ".join(last_model_explainability_monitor_execution_report_files))
else:
    last_model_explainability_monitor_execution = None
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )
    print(schedule_desc)

If there are any violations compared to the baseline, they are listed here. See [Feature Attribution Drift Violations](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-model-attribution-drift-violations.html) for the schema of the file, and how violations are detected.

In [ ]:
violations = model_explainability_monitor.latest_monitoring_constraint_violations()
if violations is not None:
    pprint.PrettyPrinter(indent=4).pprint(violations.body_dict)

By default, the analysis results are also published to CloudWatch, see [CloudWatch Metrics for Feature Attribution Drift Analysis](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-drift-cw.html).

## Cleanup

The endpoint can keep running and capturing data, but if there is no plan to collect more data or use this endpoint further, it should be deleted to avoid incurring additional charges. Note that deleting endpoint does not delete the data that was captured during the model invocations.

First stop the worker thread,

In [ ]:
invoke_endpoint_thread.terminate()

Then stop all monitors scheduled for the endpoint

In [ ]:
model_explainability_monitor.stop_monitoring_schedule()
wait_for_execution_to_finish(model_explainability_monitor)

In [ ]:
model_explainability_monitor.delete_monitoring_schedule()

Finally, delete the endpoint

In [ ]:
sagemaker_session.delete_endpoint(endpoint_name=endpoint_name)
sagemaker_session.delete_model(model_name=model_name)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Endpoint.ipynb)
